In [2]:
import pandas as pd
from pathlib import Path
from ipywidgets import widgets
from pandas_profiling import ProfileReport

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
pd.set_option('display.max_columns', 120)

In [4]:
df = pd.read_csv("C:/Users/91994/Downloads/train_yhhx1Xs/train.csv")

In [2]:
profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}},minimal = False)
profile.to_file(output_file="full.html")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 11 columns):
ID                    512087 non-null int64
Day_No                512087 non-null int64
Course_ID             512087 non-null int64
Course_Domain         512087 non-null object
Course_Type           512087 non-null object
Short_Promotion       512087 non-null int64
Public_Holiday        512087 non-null int64
Long_Promotion        512087 non-null int64
User_Traffic          512087 non-null int64
Competition_Metric    510323 non-null float64
Sales                 512087 non-null int64
dtypes: float64(1), int64(8), object(2)
memory usage: 43.0+ MB


In [6]:
labels = []
values = []
for col in df.columns:
    labels.append(col)
    values.append(df[col].isnull().sum())
    print(col, values[-1])

ID 0
Day_No 0
Course_ID 0
Course_Domain 0
Course_Type 0
Short_Promotion 0
Public_Holiday 0
Long_Promotion 0
User_Traffic 0
Competition_Metric 1764
Sales 0


In [7]:
coursedomain_dict = {'Business':0, 'Development':1, 'Finance & Accounting' :2, 'Software Marketing' : 4}
coursetype_dict = {'Course':0, 'Degree':1, 'Program':2}

df["Course_Domain"] = df["Course_Domain"].apply(lambda x: coursedomain_dict[x])
df["Course_Type"] = df["Course_Type"].apply(lambda x: coursetype_dict[x])

In [8]:
df.fillna(0, inplace=True)

In [9]:
labels = []
values = []
for col in df.columns:
    labels.append(col)
    values.append(df[col].isnull().sum())
    print(col, values[-1])

ID 0
Day_No 0
Course_ID 0
Course_Domain 0
Course_Type 0
Short_Promotion 0
Public_Holiday 0
Long_Promotion 0
User_Traffic 0
Competition_Metric 0
Sales 0


In [10]:
# Getting count features

def getCountVar(compute_df, count_df, var_name):
	grouped_df = count_df.groupby(var_name)
	count_dict = {}
	for name, group in grouped_df:
		count_dict[name] = group.shape[0]

	count_list = []
	for index, row in compute_df.iterrows():
		name = row[var_name]
		count_list.append(count_dict.get(name, 0))
	return count_list


df["ID_count"] = getCountVar(df, df, "ID")
print("ID", np.unique(df["ID_count"]))

df["Day_No_count"] = getCountVar(df, df, "Day_No")
print("Day_No", np.unique(df["Day_No_count"]))

df["Course_ID_count"] = getCountVar(df, df, "Course_ID")
print("Course_ID", np.unique(df["Course_ID_count"]))

df["CourseDomain_Count"] = getCountVar(df, df, "Course_Domain")
print("Course_Domain", np.unique(df["CourseDomain_Count"]))

df["CourseType_Count"] = getCountVar(df, df, "Course_Type")
print("Course_Type", np.unique(df["CourseType_Count"]))

df["PublicHoliday_Count"] = getCountVar(df, df, "Public_Holiday")
print("Public_Holiday", np.unique(df["PublicHoliday_Count"]))

df["ShortPromo_Count"] = getCountVar(df, df, "Short_Promotion")
print("Short_Promotion", np.unique(df["ShortPromo_Count"]))

df["LongPromo_Count"] = getCountVar(df, df, "Long_Promotion")
print("Long_Promotion", np.unique(df["LongPromo_Count"]))

ID [1]
Day_No [507 599 600]
Course_ID [698 881 882]
Course_Domain [  4410  77210 166172 264295]
Course_Type [  1764 247576 262747]
Public_Holiday [ 16202 495885]
Short_Promotion [194718 317369]
Long_Promotion [250394 261693]


In [11]:
df.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales,ID_count,Day_No_count,Course_ID_count,CourseDomain_Count,CourseType_Count,PublicHoliday_Count,ShortPromo_Count,LongPromo_Count
0,1,1,1,1,0,0,1,1,11004,0.007,81,1,599,882,264295,262747,16202,317369,250394
1,2,2,1,1,0,0,0,1,13650,0.007,79,1,600,882,264295,262747,495885,317369,250394
2,3,3,1,1,0,0,0,1,11655,0.007,75,1,600,882,264295,262747,495885,317369,250394
3,4,4,1,1,0,0,0,1,12054,0.007,80,1,600,882,264295,262747,495885,317369,250394
4,5,5,1,1,0,0,0,1,6804,0.007,41,1,600,882,264295,262747,495885,317369,250394


In [12]:
def runXGB(train_X, train_y, test_X):
        params = {}
        params["objective"] = "reg:linear"
        params["eta"] = 0.03
        params["min_child_weight"] = 10
        params["subsample"] = 0.8
        params["colsample_bytree"] = 0.7
        params["silent"] = 1
        params["max_depth"] = 10
        #params["max_delta_step"]=2
        params["seed"] = 0
        #params['eval_metric'] = "auc"
        plst = list(params.items())
        num_rounds = 1100
        xgtrain = xgb.DMatrix(train_X, label=train_y)
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        pred_test_y = model.predict(xgtest)
        return pred_test_y

In [13]:
def getPurchaseVar(compute_df, purchase_df, var_name):
        grouped_df = purchase_df.groupby(var_name)
        min_dict = {}
        max_dict = {}
        mean_dict = {}
        twentyfive_dict = {}
        seventyfive_dict = {}
        for name, group in grouped_df:
                min_dict[name] = min(np.array(group["Sales"]))
                max_dict[name] = max(np.array(group["Sales"]))
                mean_dict[name] = np.mean(np.array(group["Sales"]))
                twentyfive_dict[name] = np.percentile(np.array(group["Sales"]),25)
                seventyfive_dict[name] = np.percentile(np.array(group["Sales"]),75)

        min_list = []
        max_list = []
        mean_list = []
        twentyfive_list = []
        seventyfive_list = []
        for index, row in compute_df.iterrows():
                name = row[var_name]
                min_list.append(min_dict.get(name,0))
                max_list.append(max_dict.get(name,0))
                mean_list.append(mean_dict.get(name,0))
                twentyfive_list.append( twentyfive_dict.get(name,0))
                seventyfive_list.append( seventyfive_dict.get(name,0))

        return min_list, max_list, mean_list, twentyfive_list, seventyfive_list

In [14]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list, seventyfive_price_list = getPurchaseVar(df, df, "Day_No")
df["Day_No_MinPrice"] = min_price_list
df["Day_No_MaxPrice"] = max_price_list
df["Day_No_MeanPrice"] = mean_price_list
df["Day_No_25PercPrice"] = twentyfive_price_list
df["Day_No_75PercPrice"] = seventyfive_price_list

In [15]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list, seventyfive_price_list = getPurchaseVar(df, df, "Course_ID")
df["User_ID_MinPrice"] = min_price_list
df["User_ID_MaxPrice"] = max_price_list
df["User_ID_MeanPrice"] = mean_price_list
df["User_ID_25PercPrice"] = twentyfive_price_list
df["User_ID_75PercPrice"] = seventyfive_price_list

In [16]:
X = df.loc[:, df.columns != 'Sales']
y = df.loc[:, df.columns == 'Sales']

In [17]:
from sklearn.model_selection import train_test_split, KFold
from sklearn import ensemble
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [24]:
import xgboost as xgb
pred_test_y = runXGB(X_train, y_train, X_test)

In [28]:
pred_test_y[pred_test_y<0] = 1
test_course_id = np.array(X_test["Course_ID"])
test_Day_No = np.array(X_test["Day_No"])
test_ID = np.array(X_test["ID"])
out_df = pd.DataFrame({"ID": test_ID, "Course_ID":test_course_id, "Day_No": test_Day_No})

out_df["Purchase"] = pred_test_y
out_df.to_csv("sub20.csv", index=False)